# 量化子消去のアルゴリズム(具体的な命題文の場合)

具体的に次の命題文について，量化子消去を行う．

$\forall x (x^2 + a^2 < 1 \land a < x)$

この命題文は, $-1<a<1/\sqrt{2}$に同値である．

この命題文に登場する多項式は$x^2 + a^2 - 1, a-x$のみである．

In [40]:
R.<x,a> = QQ[]

# # ここにCADしたい多項式のリストを渡す
F = [x^2 + a^2-1,a-x];F

[x^2 + a^2 - 1, -x + a]

量化子消去を行うためにCADを求める．まず，射影$G= \mathrm{PROJ}(F)$をを求める．

In [41]:
# 主部分終結式係数全体(principal subresultant coefficient)
# 一つ一つ出力するほうがわかりやすいが, sagemathはsubresultantsモジュールしか備えておらず, そのためすべてを出力するように実装する.
def PSCs(f, g):
    """
    deg(f)>= 0, deg(g)>= 0, つまり, f != 0, g!= 0を仮定する.
    """
    if f == 0 or g == 0:
        return False
    psc_list = []
    subres = f.subresultants(g)
    for i in range(len(subres)):
        if subres[i].degree()==i:
            psc_list.append(subres[i].lc())
        else:
            psc_list.append(0)
    
    return psc_list

In [42]:
def PROJ(F):
    R = F[0].parent()
    F = [R(f).polynomial(R.gen()) for f in F]
    
    # FのReductionをすべて計算し, その集合をBとする
    B = []
    for f in F:
        A = f
        while(A != 0):
            B.append(A)
            A = A - A.lt()
            
    # Bの先頭項係数全体をLとする.
    L = {b.lc() for b in B}

    # PSC_k(b, b')全体をS1とする.
    S1 = []
    for b in B:
        if b.degree()>0:
            S1 = S1 + PSCs(b, b.diff())            
    S1 = set(S1)
    
    # PSC_k(b_1, b_2)全体をS2とする.
    S2 = []
    for i in range(len(B)):
        for j in range(i+1,len(B)):
            S2 = S2 + PSCs(B[i],B[j])
    S2 = set(S2)
    
    proj = {f for f in L | S1 | S2 if not f in RR}
    return list(proj)

In [43]:
G = PROJ(F);G

[a^2 - 1, a, 4*a^2 - 4, 2*a^2 - 1]

$G$不変な$\mathbb{R}$の分割を求める．

In [30]:
def base(F):
    # 前処理: 定数となるようなFは除く.
    F = [f for f in F if f not in AA]
    
    B = []
    for g in F:
        B += g.roots(AA, multiplicities = False)
    B = list(set(B))
    B.sort()
    
    ans = []
    if(len(B) == 0):
        ans.append(AA(0))
    else:
        ans.append(B[0]-1)
        for i in range(len(B)-1):
            ans.append(B[i])
            ans.append((B[i]+B[i+1])/2)
        ans.append(B[-1])
        ans.append(B[-1]+1)
    
    ans = [(a,) for a in ans]
    return ans


In [31]:
base(G)

[(-2,),
 (-1,),
 (-0.8535533905932738?,),
 (-0.7071067811865475?,),
 (-0.3535533905932738?,),
 (0,),
 (0.3535533905932738?,),
 (0.7071067811865475?,),
 (0.8535533905932738?,),
 (1,),
 (2,)]

ここで求めた$\mathbb{R}$の分割の各セルの定義式が問題となる．
ここで，$G$の生成する微分で閉じた集合$AG$とすると, これは各セルを分離する定義式である．

In [36]:
f = x^2 + a^2
f = R(f).polynomial(a)
f.diff()

2*a

In [48]:
def Augment(F):
    R = F[0].parent()
    F = [R(f).polynomial(R.gen()) for f in F]
    
    AF = []
    for f in F:
        while(f not in AA):
            AF.append(f)
            f = f.diff()
    
    AF = set(AF)
    return AF

In [50]:
AG = Augment(G);AG

{a, 2*a, 4*a, 8*a, a^2 - 1, 2*a^2 - 1, 4*a^2 - 4}

この集合$AG$は, 上で求めた$\mathbb{R}$の定義式となる．

In [51]:
def lifting(Base, F):
    R = F[0].parent()
    ans = []
    for b in Base:
        F_1 = [AA[f"{R.gen()}"](f((R.gen(),)+b)) for f in F]
        for a in base(F_1):
            ans.append(a+b)
    return ans

持ち上げを計算し, どのセルが条件式を満たしているかをチェックする．
条件を満たすセルの代表点をすべて表示する．

In [73]:
G = PROJ(F)
Base = base(G)
print(Base)
Samples = lifting(Base,F)

[(-2,), (-1,), (-0.8535533905932738?,), (-0.7071067811865475?,), (-0.3535533905932738?,), (0,), (0.3535533905932738?,), (0.7071067811865475?,), (0.8535533905932738?,), (1,), (2,)]


In [80]:
# 条件を満たしているサンプルポイントのリスト:
Satisfies = []
for (b1,) in Base:
    for (a,b2) in Samples:
        if b1 == b2:
            if F[0](a,b2)<0 and F[1](a,b2)<0:
                Satisfies.append((b1,))
                break
Satisfies

[(-0.8535533905932738?,),
 (-0.7071067811865475?,),
 (-0.3535533905932738?,),
 (0,),
 (0.3535533905932738?,)]

最後に，代数的命題文で記述する．簡単のため，AGは先頭項係数を0にしておく.

In [88]:
AG = {g/g.lc() for g in AG};AG

{a, a^2 - 1, a^2 - 1/2}

In [98]:
sentences = []
for (b,) in Satisfies:
        lst = []
        for g in AG:
            if g(b) > 0:
                lst += [f"{g} > 0"]
            elif g(b) == 0:
                lst += [f"{g} = 0"]
            elif g(b) < 0:
                lst += [f"{g} < 0"]
        sentences.append("(" + " and ".join(lst) + ")")

sentences       

['(a^2 - 1 < 0 and a < 0 and a^2 - 1/2 > 0)',
 '(a^2 - 1 < 0 and a < 0 and a^2 - 1/2 = 0)',
 '(a^2 - 1 < 0 and a < 0 and a^2 - 1/2 < 0)',
 '(a^2 - 1 < 0 and a = 0 and a^2 - 1/2 < 0)',
 '(a^2 - 1 < 0 and a > 0 and a^2 - 1/2 < 0)']

これらをorでつなげば, (一応)量化子消去が終了したことになる.たしかにこれは$-1 < a < 1/\sqrt{2}$ではあるけども...．